In [14]:
# Dependencies
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [3]:
movies_df = pd.read_csv('resources/IMDb_movies.csv', encoding='utf8')

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
cleaned_movies_df = movies_df.drop(columns=['date_published', 'description', 'original_title', 'duration', 'language', 'actors', 'avg_vote', 'votes', 'budget', 'usa_gross_income', 'worlwide_gross_income', 'metascore'])
cleaned_movies_df

,imdb_title_id,title,year,genre,country,director,writer,production_company,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,1894,Romance,USA,Alexander Black,Alexander Black,Alexander Black Photoplays,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",Australia,Charles Tait,Charles Tait,J. and N. Tait,7.0,7.0
2,tt0001892,Den sorte drøm,1911,Drama,"Germany, Denmark",Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,5.0,2.0
3,tt0002101,Cleopatra,1912,"Drama, History",USA,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,25.0,3.0
4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",Italy,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,2020,Comedy,"France, Belgium",Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour",Monkey Pack Films,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",Netherlands,Johan Nijenhuis,"Radek Bajgar, Herman Finkers",Johan Nijenhuis & Co,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,2019,Drama,India,Vineesh Aaradya,"Vineesh Aaradya, Vineesh Aaradya",RMCC Productions,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,2019,"Drama, Family",Turkey,Ahmet Faik Akinci,"Ahmet Faik Akinci, Kasim Uçkan",Gizem Ajans,NaN,NaN


In [5]:
# Read oscars awards csv
oscars_df = pd.read_csv('resources/the_oscar_award.csv', encoding='utf8')
oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [6]:
# Filter Categories to be used
cleaned_oscars_df = oscars_df[(oscars_df['winner']==True) &
                              ((oscars_df['category']=='BEST PICTURE') | 
                               (oscars_df['category']=='DIRECTING') | 
                               (oscars_df['category']=='BEST MOTION PICTURE') | 
                               (oscars_df['category']=='CINEMATOGRAPHY (Black-and-White)') |
                               (oscars_df['category']=='CINEMATOGRAPHY (Color)'))]
cleaned_oscars_df

,year_film,year_ceremony,ceremony,category,name,film,winner
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True
...,...,...,...,...,...,...,...
10095,2017,2018,90,BEST PICTURE,"Guillermo del Toro and J. Miles Dale, Producers",The Shape of Water,True
10180,2018,2019,91,DIRECTING,Alfonso Cuarón,Roma,True
10219,2018,2019,91,BEST PICTURE,"Jim Burke, Charles B. Wessler, Brian Currie, P...",Green Book,True
10306,2019,2020,92,DIRECTING,Bong Joon Ho,Parasite,True


In [7]:
# Add movie ID to oscars movies
%timeit 
cleaned_oscars_df['title_id']=""

for index, film in cleaned_oscars_df.iterrows():
    for index2, title in cleaned_movies_df.iterrows():
            if (str(film['film']).lower() == str(title['title']).lower()):
                cleaned_oscars_df.loc[index, 'title_id'] = title['imdb_title_id']
cleaned_oscars_df

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,year_film,year_ceremony,ceremony,category,name,film,winner,title_id
59,1928,1929,2,DIRECTING,Frank Lloyd,The Divine Lady,True,
98,1929,1930,3,DIRECTING,Lewis Milestone,All Quiet on the Western Front,True,
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047
...,...,...,...,...,...,...,...,...
10095,2017,2018,90,BEST PICTURE,"Guillermo del Toro and J. Miles Dale, Producers",The Shape of Water,True,
10180,2018,2019,91,DIRECTING,Alfonso Cuarón,Roma,True,tt6155172
10219,2018,2019,91,BEST PICTURE,"Jim Burke, Charles B. Wessler, Brian Currie, P...",Green Book,True,tt6966692
10306,2019,2020,92,DIRECTING,Bong Joon Ho,Parasite,True,tt6751668


In [8]:
cleaned_movies_df.to_csv('output/cleaned_imdb.csv')
cleaned_oscars_df.to_csv('output/oscarswithid.csv')

In [22]:
cleaned2_oscars_df = cleaned_oscars_df.replace('', np.NaN)
cleaned2_oscars_df = a.dropna(how='any')
cleaned2_oscars_df

,year_film,year_ceremony,ceremony,category,name,film,winner,title_id
139,1930,1931,4,DIRECTING,Norman Taurog,Skippy,True,tt0163246
171,1931,1932,5,DIRECTING,Frank Borzage,Bad Girl,True,tt3598108
234,1932,1933,6,DIRECTING,Frank Lloyd,Cavalcade,True,tt0427047
360,1935,1936,8,DIRECTING,John Ford,The Informer,True,tt0020025
1503,1943,1944,16,DIRECTING,Michael Curtiz,Casablanca,True,tt10497826
...,...,...,...,...,...,...,...,...
9970,2016,2017,89,BEST PICTURE,"Adele Romanski, Dede Gardner and Jeremy Kleine...",Moonlight,True,tt4975722
10180,2018,2019,91,DIRECTING,Alfonso Cuarón,Roma,True,tt6155172
10219,2018,2019,91,BEST PICTURE,"Jim Burke, Charles B. Wessler, Brian Currie, P...",Green Book,True,tt6966692
10306,2019,2020,92,DIRECTING,Bong Joon Ho,Parasite,True,tt6751668


In [24]:
cleaned2_oscars_df.to_csv('output/final_oscars.csv')

In [ ]:
# cleaned_oscars_df = oscars_df.drop(columns=['ceremony', ])
cleaned_oscars_df = oscars_df[(oscars_df['winner']==True) &
                              (oscars_df['category']=='BEST PICTURE')]
cleaned_oscars_df

In [ ]:
cleaned_movies_df = cleaned_movies_df.drop(columns=['awarded'])
cleaned_movies_df

In [ ]:
cleaned_movies_df['awarded'] = ""
for index, title in cleaned_movies_df.iterrows():
#     print(title['title'])
    
    for index2, film in cleaned_oscars_df.iterrows():
#         print(film['film'])
        if str(title['title']).lower() == str(film['film']).lower():
            cleaned_movies_df.loc[index, 'awarded'] = True
#             print(f"imdb: {title} / oscar: {film}")
            
            
#         else:
#             copy_movies_df['Awarded'] = False
        

In [ ]:
cleaned_movies_df[cleaned_movies_df['awarded']==True]

In [ ]:
oscars_df[(oscars_df['winner']==True) &
                    (oscars_df['category']==('BEST PICTURE'))]

In [49]:
directors_df = cleaned_movies_df['director'].str.split(',', n=5, expand = True)
# directors_df2 = directors_df.copy()
# directors_df2.dropna(how='any')
director1_df = pd.DataFrame(directors_df[0])
director1_df = director1_df.rename(columns={0:'name'})
director2_df = pd.DataFrame(directors_df[1])
director2_df = director2_df.dropna(how='any')
director2_df = director2_df.rename(columns={1:'name'})
director2_df


,name
4,Adolfo Padovan
8,Grigore Brezeanu
9,James Keane
18,Stellan Rye
20,Eleuterio Rodolfi
...,...
85807,Tonni Zinck
85812,Steve Noir
85814,Erik Verkerk
85835,Syed Nurfaiz Khalid bin Syed Ibrahim


In [62]:
# Concat dataframes
cleaned_directors = pd.concat([director1_df, director2_df], ignore_index = True)
cleaned_directors = pd.DataFrame(cleaned_directors['name'].unique())
cleaned_directors = cleaned_directors.rename(columns={0:'name'})
cleaned_directors

,name
0,Alexander Black
1,Charles Tait
2,Urban Gad
3,Charles L. Gaskill
4,Francesco Bertolini
...,...
36590,Geethika Sudip
36591,Steve Noir
36592,Erik Verkerk
36593,Syed Nurfaiz Khalid bin Syed Ibrahim


In [64]:
# export directors dataframe to csv
cleaned_directors.to_csv('output/directors.csv')

In [66]:
# Oscars categories
cleaned_categories_df = pd.DataFrame(cleaned_oscars_df['category'].unique())
cleaned_categories_df = cleaned_categories_df.rename(columns={0:'categories'})
cleaned_categories_df

,categories
0,DIRECTING
1,CINEMATOGRAPHY (Black-and-White)
2,CINEMATOGRAPHY (Color)
3,BEST MOTION PICTURE
4,BEST PICTURE


In [67]:
# Export oscars categories
cleaned_categories_df.to_csv('output/oscars_categories.csv')